In [1]:
import importlib
from collections import defaultdict, deque

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from kaggle_environments import make

%matplotlib inline

Loading environment football failed: No module named 'gfootball'


In [2]:
model_module = importlib.import_module("handyrl.model")
env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")

In [3]:
e = env_module.Environment()
e.reset()

In [4]:
obs = e.observation()

In [5]:
field = obs[8:13].sum(0)
field

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [6]:
def around(a):
    a_ = [
        ((a[0] - 1) % 7, a[1]),
        ((a[0] + 1) % 7, a[1]),
        (a[0], (a[1] - 1) % 11),
        (a[0], (a[1] + 1) % 11),
    ]
    return [e for e in a_ if field[e[0], e[1]] == 0]

In [7]:
heads = np.where(obs[0] == 1)

for r, c in zip(heads[0], heads[1]):
    break

print(f"{r}, {c}")

6, 4


In [8]:
v = around((r, c))
v

[(5, 4), (0, 4), (6, 3), (6, 5)]

In [9]:
def bfs(v):
    q = deque(v)
    movable = np.zeros([7, 11])
    searched = defaultdict(bool)
    while len(q) != 0:
        v = q.popleft()
        movable[v] = 1
        searched[v] = True
        edges = [a for a in around(v) if not searched[a]]
        for edge in edges:
            q.append(edge)
    return movable

In [10]:
bfs(v)

array([[1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.]])